In [358]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import one_hot, Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Embedding,Conv1D,GlobalMaxPooling1D,MaxPooling1D,Dense,AdditiveAttention,MultiHeadAttention,Bidirectional,LSTM,Flatten,Cropping3D,UpSampling3D,Attention,Dropout,Input

In [359]:
# importing data
data=pd.read_csv('/kaggle/input/stock-data/stock_data.csv')

In [360]:
#checking for null data
print(data['Text'].isnull().sum())
print(data['Sentiment'].isnull().sum())
data

0
0


,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1
...,...,...
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1


In [361]:
data.isnull().values.any()

False

In [362]:
data['Text'][1]

'user: AAP MOVIE. 55% return for the FEA/GEED indicator just 15 trades for the year.  AWESOME.  '

In [363]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Removes HTML tags: replaces anything between opening and closing <> with empty space'''

    return TAG_RE.sub('', text)

In [364]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [365]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''
    
    sentence = sen.lower()

    # Remove html tags
    sentence = remove_tags(sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [366]:
X = []
sentences = list(data['Text'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [367]:
print(X[0])
print(data['Text'][0])

kickers watchlist xide tit soq pnk cpw bpz aj trade method method see prev posts
Kickers on my watchlist XIDE TIT SOQ PNK CPW BPZ AJ  trade method 1 or method 2, see prev posts


In [368]:
data['Sentiment']=data['Sentiment'].map({-1:0,1:1})

In [369]:
# x=np.array(X)
y=np.array(data['Sentiment'])
x=X

In [370]:
x_train,x_test,y_train,y_test=train_test_split(x,y)
y_train.shape,y_test.shape

((4343,), (1448,))

In [371]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(x_train)

x_train = word_tokenizer.texts_to_sequences(x_train)
x_test = word_tokenizer.texts_to_sequences(x_test)

In [372]:
vocab_length = len(word_tokenizer.word_index) +1

vocab_length


7707

In [373]:
maxlen = 50

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

In [374]:
x_train.shape,y_train.shape

((4343, 50), (4343,))

In [375]:
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/kaggle/input/kaggleinputglovedataglove6b200dtxt/a2_glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [376]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [377]:
embedding_matrix.shape

(7707, 100)

In [378]:
embedding_input = Input(shape=(maxlen,))
embedding_output=Embedding(vocab_length,100,weights=[embedding_matrix], input_length=maxlen)(embedding_input)
CNN_output=Conv1D(256,3,padding ="same",activation="relu")(embedding_output)
maxpool_output=MaxPooling1D()(CNN_output)
dropout_output=Dropout(0.2)(maxpool_output)
CNN_output=Conv1D(512,5,padding="same",activation="relu")(dropout_output)
# maxpool_output=MaxPooling1D()(CNN_output)
dropout_output=Dropout(0.2)(CNN_output)
# CNN_output=Conv1D(256,7,padding="same",activation="relu")(dropout_output)
# BiLSTM_output=Bidirectional(LSTM(128, return_sequences=True))(dropout_output)
BiLSTM_output=Bidirectional(LSTM(256))(dropout_output)
attention_output = Attention()([BiLSTM_output, BiLSTM_output])
l=Dense(units=1,activation="sigmoid")(attention_output)

In [379]:
model = tf.keras.Model(inputs=[embedding_input], outputs = [l])

In [380]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 embedding_13 (Embedding)       (None, 50, 100)      770700      ['input_14[0][0]']               
                                                                                                  
 conv1d_23 (Conv1D)             (None, 50, 256)      77056       ['embedding_13[0][0]']           
                                                                                                  
 max_pooling1d_9 (MaxPooling1D)  (None, 25, 256)     0           ['conv1d_23[0][0]']              
                                                                                           

In [381]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [382]:
model.fit(x_train,y_train,epochs=15,batch_size=64,validation_data=(x_test,y_test))

Epoch 1/15
68/68 [==============================] - 19s 208ms/step - loss: 0.6694 - accuracy: 0.6284 - val_loss: 0.6552 - val_accuracy: 0.6464
Epoch 2/15
68/68 [==============================] - 7s 102ms/step - loss: 0.6586 - accuracy: 0.6330 - val_loss: 0.6496 - val_accuracy: 0.6464
Epoch 3/15
68/68 [==============================] - 6s 79ms/step - loss: 0.6581 - accuracy: 0.6330 - val_loss: 0.6510 - val_accuracy: 0.6464
Epoch 4/15
68/68 [==============================] - 3s 47ms/step - loss: 0.6547 - accuracy: 0.6330 - val_loss: 0.6120 - val_accuracy: 0.6464
Epoch 5/15
68/68 [==============================] - 1s 22ms/step - loss: 0.5663 - accuracy: 0.6912 - val_loss: 0.5254 - val_accuracy: 0.7424
Epoch 6/15
68/68 [==============================] - 2s 32ms/step - loss: 0.4012 - accuracy: 0.8280 - val_loss: 0.5073 - val_accuracy: 0.7673
Epoch 7/15
68/68 [==============================] - 2s 35ms/step - loss: 0.2629 - accuracy: 0.9077 - val_loss: 0.5450 - val_accuracy: 0.7804
Epoch 8/15

In [383]:
# prediction=model.predict(x_test)
y_test_pred=model.predict(x_test)
model.evaluate(x_test,y_test)



46/46 [==============================] - 0s 5ms/step - loss: 0.8170 - accuracy: 0.7825


[0.81695556640625, 0.7824585437774658]

In [384]:
y_test_pred
threshold = 0.5

# Convert the data to 1 and 0 using the threshold
binary_data = (y_test_pred >= threshold).astype(int)

print(binary_data)
print(y_test_pred)
y_test_pred=binary_data

[[1]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
[[0.99770725]
 [0.34223723]
 [0.01113324]
 ...
 [0.99472505]
 [0.9529896 ]
 [0.9565519 ]]


In [385]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_test_pred))


              precision    recall  f1-score   support

           0       0.71      0.64      0.68       512
           1       0.81      0.86      0.84       936

    accuracy                           0.78      1448
   macro avg       0.76      0.75      0.76      1448
weighted avg       0.78      0.78      0.78      1448

